In [1]:
import os

In [2]:
%pwd

'd:\\Projects\\End_to_End_MLProject_with-MLFLOW\\research'

In [3]:
os.chdir("../")

In [4]:
%pwd

'd:\\Projects\\End_to_End_MLProject_with-MLFLOW'

In [5]:
from dataclasses import dataclass
from pathlib import Path

In [6]:
@dataclass(frozen= True)
class ModelTrainerConfig:
    root_dir: Path
    train_data_path: Path
    test_data_path: Path
    model_name: str
    alpha: float
    l1_ratio: float
    target_column: str

In [7]:
from src.MLProject_With_MLFLOW.constants import *
from src.MLProject_With_MLFLOW.utils.common import read_yaml, create_directory

In [8]:
class ConfigurationManager:
    def __init__(self,
                 config_filepath = CONFIG_FILE_PATH,
                 schema_filepath = SCHEMA_FILE_PATH,
                 params_filepath = PARAMS_FILE_PATH):
        
        self.config = read_yaml(config_filepath)
        self.schema = read_yaml(schema_filepath)
        self.params = read_yaml(params_filepath)


        create_directory([self.config.artifacts_root])


    def get_model_trainer(self) -> ModelTrainerConfig:

        config = self.config.model_trainer
        params = self.params.ElasticNet
        schema = self.schema.TARGET_COLUMN

        create_directory([config.root_dir])

        model_trainer_config = ModelTrainerConfig(
            root_dir = config.root_dir,
            train_data_path = config.train_data_path,
            test_data_path = config.test_data_path,
            model_name = config.model_name,
            alpha = params.alpha,
            l1_ratio = params.l1_ratio,
            target_column = schema.target_column

        )

        return model_trainer_config

In [9]:
import os
import pandas as pd
from src.MLProject_With_MLFLOW.utils.common import logger
from sklearn.linear_model import ElasticNet
import joblib

In [10]:
class ModelTraining:
    def __init__(self,config: ModelTrainerConfig):
        self.config = config



    def train(self):
        train_data = pd.read_csv(self.config.train_data_path)
        test_data = pd.read_csv(self.config.test_data_path)


        train_x = train_data.drop([self.config.target_column],axis = 1)
        test_x = test_data.drop([self.config.target_column],axis=1)
        train_y = train_data[self.config.target_column]
        test_y = test_data[self.config.target_column]


        model = ElasticNet(alpha= self.config.alpha, l1_ratio= self.config.l1_ratio, random_state=42)
        model.fit(train_x, train_y)



        joblib.dump(model, os.path.join(self.config.root_dir, self.config.model_name))

In [12]:
try:
    config = ConfigurationManager()
    model_trainer_config = config.get_model_trainer()
    model_trainer = ModelTraining(config= model_trainer_config)
    model_trainer.train()

except Exception as e :
    raise e

[2023-11-01 13:11:47,405 - INFO - common - yaml_file config\config.yaml loaded successfully]
[2023-11-01 13:11:47,413 - INFO - common - yaml_file schema.yaml loaded successfully]
[2023-11-01 13:11:47,420 - INFO - common - yaml_file params.yaml loaded successfully]
[2023-11-01 13:11:47,421 - INFO - common - created directory atartifacts]
[2023-11-01 13:11:47,425 - INFO - common - created directory atartifacts/model_trainer]
